## Charting Template


In [29]:
# Import Libraries
import pandas as pd
import numpy as np
import urllib
import requests
import riskfolio as rp
from datetime import datetime, timedelta
from copy import deepcopy

from highcharts import Highchart
from chart_builder import plot_chart, CHART_DEFAULTS
from IPython.display import display, HTML, display_html
import import_ipynb

from pricing_engine.engine import historical_prices, price_ondate
from backend.utils import pickle_it, jformat

# Chart libraries + settings
# plt.style.use('seaborn-whitegrid')
pd.options.mode.chained_assignment = None  # default='warn' - disable some pandas warnings
pd.options.display.float_format = '{:,.2f}'.format


### Chart Template

In [30]:
%%capture
# MatPlotLib settings
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from cycler import cycler

# Swan Colors
swan_colors = ['#31597F', '#7B96B0', '#C3D9E9', '#DAE1E8']
mpl.rcParams['lines.color'] = '#31597F'
mpl.rcParams['text.color'] = '#31597F'
mpl.rcParams['axes.prop_cycle'] = cycler(color=swan_colors)
mpl.rcParams['lines.linewidth'] = 1
sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rcParams["figure.figsize"] = (10,5)
plt.rc('axes', titlesize=12)
plt.rc('axes', labelsize=10)   
plt.rc('xtick', labelsize=10, color='#00305E')  
plt.rc('ytick', labelsize=10, color='#00305E')    
plt.rc('legend', fontsize=12)
plt.rc('font', size=8)          

import matplotlib.ticker as ticker

# Define the formatter
formatter = ticker.StrMethodFormatter('{x:,.0f}')

from PIL import Image
fig, ax = plt.subplots()
img = Image.open('swan.jpg')
width, height = ax.figure.get_size_inches()*fig.dpi
wm_width = int(width / 2) 
scaling = (wm_width / float(img.size[0]))
wm_height = int(float(img.size[1])*float(scaling))
img = img.resize((wm_width, wm_height), Image.ANTIALIAS)

def add_watermark(ax, img):
    ax.imshow(img, aspect='equal', zorder=1, alpha=0.2, extent=(0.3, 1.6, 0.05, 0.3), transform=ax.transAxes)
    


## Import the altcoin list. 
### ⚠️ Please wear gloves and a mask

In [31]:
# Large data set - retrieve and save locally
# url = 'https://min-api.cryptocompare.com/data/all/coinlist'
# response = requests.get(url)
# data = response.json()
# pickle_it('save', 'altcoins.pkl', data)

data = pickle_it('load', 'altcoins.pkl')

In [32]:
# Load the data into a pandas DataFrame
altcoin_df = pd.DataFrame(data['Data'])
altcoin_df = altcoin_df.T
altcoin_df['ContentCreatedOn'] = pd.to_datetime(altcoin_df['ContentCreatedOn'], unit='s')

In [33]:
# Increment n number of months of certain date
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1])
    new_date = (date.replace(day=d,month=m, year=y))
    return new_date


def add_periods(date, periods, frequency):
    if frequency.upper() == 'D' or 'DAY' in frequency.upper():
        return (date + timedelta(days=periods))
    if frequency.upper() == 'W' or 'WEEK' in frequency.upper():
        return (date + timedelta(days=periods * 7))
    if frequency.upper() == 'M' or 'MONTH' in frequency.upper():
        return(monthdelta(date, periods))
    if frequency.upper() == 'Y' or 'YEAR' in frequency.upper():
        return(monthdelta(date, periods * 12))

# Create a function that given a df that will return returns in 
# 1yr, 2yr, 3yr, etc. after a certain start date
def calculate_returns(ticker, 
                      benchmark='BTC', 
                      max_years=7, 
                      source=['cryptocompare']):
    
    from requests.exceptions import ConnectionError
    try:
        tmp_df = historical_prices(ticker, source=source)
    except ConnectionError:
        print(f"!!!!!  WARNING: Could not connect to API for ticker: {ticker}")
        return None
            
    if tmp_df.empty == True:
        return None
    current_date = tmp_df.index.min()
    years = range(1, max_years + 1)
    return_dict = {}
    for year in years:
        return_dict[year] = {}
        return_dict[year]['year'] = year
        return_dict[year]['ticker'] = ticker
        return_dict[year]['benchmark'] = benchmark
        return_dict[year]['start_date'] = current_date
        return_dict[year]['start_price'] = price_ondate(ticker, current_date, source=source).close[0]
        return_dict[year]['start_price_benchmark'] = price_ondate(benchmark, current_date, source=source).close[0]
        # End of period now
        current_date = add_periods(current_date, 1, 'Y')
        return_dict[year]['end_date'] = current_date
        return_dict[year]['end_price'] = price_ondate(ticker, current_date, source=source).close[0]
        return_dict[year]['end_price_benchmark'] = price_ondate(benchmark, current_date, source=source).close[0]
        return_dict[year]['factor_period'] = (
            return_dict[year]['end_price'] / 
            return_dict[year]['start_price']) 
        return_dict[year]['return_period'] = (return_dict[year]['factor_period']) - 1 
        return_dict[year]['factor_period_benchmark'] = (
            return_dict[year]['end_price_benchmark'] / 
            return_dict[year]['start_price_benchmark']) 
        return_dict[year]['status'] = 'Data Available'
    
    return(return_dict)


In [36]:
# Download all data
ticker_list = altcoin_df['Symbol'].to_list()
from pricing_engine.engine import download_data
download_data(ticker_list)

In [38]:
# Let's run analysis for each of the coins and merge into the df

# Create a Df and include BTC data
tmp = calculate_returns('BTC')
ret_df = pd.DataFrame(tmp).T

count = 0
# Get all the coins' data
for index,row in altcoin_df.iterrows():
    count += 1
    print("Getting data for " + row['Symbol'] + " - " + str(count) + " / " + str(len(altcoin_df)))
    data = calculate_returns(str(row['Symbol']))
    if data == None:
        status = 'No Pricing Data. Probably Dead.'
        dict_ret = {'status': [status],
                    'ticker': [row['Symbol']]}
        ret_df = pd.concat([ret_df, pd.DataFrame(dict_ret)])
    else:
        symbol_df = pd.DataFrame(data).T
        symbol_df['factor_period'] = symbol_df['factor_period'].fillna(0)
        symbol_df['factor_period_benchmark'] = symbol_df['factor_period_benchmark'].fillna(0)
        symbol_df['over_benchmark'] = symbol_df['factor_period'] / symbol_df['factor_period_benchmark']
        symbol_df['cum_outperformance'] = symbol_df['over_benchmark'].cumprod()
        symbol_df['cum_outperformance'] = symbol_df['cum_outperformance'].fillna(0)
        ret_df = pd.concat([ret_df, symbol_df])
    
ret_df
    

Getting data for 42 - 1 / 8451
Getting data for 300 - 2 / 8451
Getting data for 365 - 3 / 8451
Getting data for 404 - 4 / 8451
Getting data for 433 - 5 / 8451
Getting data for 611 - 6 / 8451
Getting data for 808 - 7 / 8451
Getting data for 888 - 8 / 8451
Getting data for 1337 - 9 / 8451
Getting data for 1717 - 10 / 8451
Getting data for 2015 - 11 / 8451
Getting data for BTCD - 12 / 8451
Getting data for CRAIG - 13 / 8451
Getting data for XBS - 14 / 8451
Getting data for XPY - 15 / 8451
Getting data for PRC - 16 / 8451
Getting data for YBC - 17 / 8451
Getting data for DANK - 18 / 8451
Getting data for GIVE - 19 / 8451
Getting data for KOBO - 20 / 8451
Getting data for DT - 21 / 8451
Getting data for CETI - 22 / 8451
Getting data for SUP - 23 / 8451
Getting data for XPD - 24 / 8451
Getting data for GEO - 25 / 8451
Getting data for CHASH - 26 / 8451
Getting data for NXTI - 27 / 8451
Getting data for WOLF - 28 / 8451
Getting data for XDP - 29 / 8451
Getting data for AC - 30 / 8451
Getting 

,year,ticker,benchmark,start_date,start_price,start_price_benchmark,end_date,end_price,end_price_benchmark,factor_period,return_period,factor_period_benchmark,status,over_benchmark,cum_outperformance
1,1,BTC,BTC,2010-07-17,0.05,0.05,2011-07-17,13.16,13.16,265.80,264.80,265.80,Data Available,NaN,NaN
2,2,BTC,BTC,2011-07-17,13.16,13.16,2012-07-17,8.80,8.80,0.67,-0.33,0.67,Data Available,NaN,NaN
3,3,BTC,BTC,2012-07-17,8.80,8.80,2013-07-17,98.50,98.50,11.19,10.19,11.19,Data Available,NaN,NaN
4,4,BTC,BTC,2013-07-17,98.50,98.50,2014-07-17,626.58,626.58,6.36,5.36,6.36,Data Available,NaN,NaN
5,5,BTC,BTC,2014-07-17,626.58,626.58,2015-07-17,279.62,279.62,0.45,-0.55,0.45,Data Available,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,3,ZB,BTC,2020-06-04,0.23,"9,794.55",2021-06-04,0.74,"36,861.50",3.15,2.15,3.76,Data Available,0.84,0.22
4,4,ZB,BTC,2021-06-04,0.74,"36,861.50",2022-06-04,0.13,"29,845.48",0.18,-0.82,0.81,Data Available,0.22,0.05
5,5,ZB,BTC,2022-06-04,0.13,"29,845.48",2023-06-04,0.45,"26,736.21",3.42,2.42,0.90,Data Available,3.82,0.18
6,6,ZB,BTC,2023-06-04,0.45,"26,736.21",2024-06-04,0.45,"26,736.21",1.00,0.00,1.00,Data Available,1.00,0.18


In [ ]:

# xls_df = ret_df.loc[ret_df['ticker'] == '365']
# xls_df.to_excel('tmp.xls')

# yr = 5
# ticker = 'YBC'
# ret_df.loc[((ret_df['ticker'] == ticker))]

# performance = float(ret_df.loc[((ret_df['ticker'] == ticker) & (ret_df['year'] == yr))]['cum_outperformance'])
# performance

In [51]:
# Format the data for Sankey Diagram
max_years = ret_df['year'].max()
yrs = range(0, max_years + 1)

# If the coin has a 85% drop in BTC terms, it's considered dead
# May the rekt spirits be with the bag holders
death_threshold = 0.15

# Build the data for Sankey Diagram
label = []
customdata = []
sourcedata = []
targetdata = []
valuedata = []

# Create all Labels
label_finder = {}
for yr in yrs:
    if yr == 0:
        label.append('Launch | Year 0')
        label_finder[yr] = {}
        label_finder[yr]['out'] = len(label) - 1
        label_finder[yr]['under'] = len(label) - 1
        label_finder[yr]['died'] = len(label) - 1
    else:
        label_finder[yr] = {}
        label.append(f'Outperforming BTC | Year {str(yr)}')
        label_finder[yr]['out'] = len(label) - 1
        label.append(f'Underperforming BTC | Year {str(yr)}')
        label_finder[yr]['under'] = len(label) - 1
        label.append(f'Died. RIP. | Year {str(yr)}')
        label_finder[yr]['died'] = len(label) - 1

for index,row in altcoin_df.iterrows():
    source = 0 # First label
    ticker = row['Symbol']
    for yr in yrs:
        if yr == 0:
            continue
        dead = False
        # Check if it's outperforming or not
        # Get relevant data
        try:
            performance = float(ret_df.loc[((ret_df['ticker'] == ticker) & (ret_df['year'] == yr))]['cum_outperformance'])
        except Exception:
            performance = 0 # RIP
        # print(ticker, yr)
        # print ("check the label...")
        # print (performance)
        if dead == False and performance > 1.00:
            target = label_finder[yr]['out'] 
            # print (label[target])
            customdata.append(f"{ticker} on year {str(int(yr))} is valued at {jformat(str(performance),2)}x of BTC")
        elif dead == False and performance < 1.00 and performance > death_threshold:
            target = label_finder[yr]['under'] 
            # print (label[target])
            customdata.append(f"{ticker} on year {str(int(yr))} is valued at {jformat(str(performance),2)}x of BTC")
            
            customdata.append(f"{ticker} on year {str(int(yr))} is valued at {jformat(str(performance),2)}x of BTC")
        elif performance < death_threshold or dead == True:
            target = label_finder[yr]['died'] 
            # print (label[target])
            dead = True
            customdata.append(f"{ticker} on year {str(int(yr))} is dead. RIP.")
            
        # Append data to lists
        sourcedata.append(source)
        targetdata.append(target)
        valuedata.append(1) # every coin == 1 so it's a count
                
        # Save the label for next loop
        source = target

        

In [52]:
# These all have to match the same length
print(len(label))
print(len(customdata))
print(len(sourcedata))
print(len((targetdata)))
print(len(valuedata))

22
66426
59157
59157
59157


In [ ]:
import plotly.graph_objects as go

def nodify(node_names):
    node_names = label
    # uniqe name endings
    ends = sorted(list(set([e[-1] for e in node_names])))
    
    # intervals
    steps = 1/len(ends)

    # x-values for each unique name ending
    # for input as node position
    nodes_x = {}
    xVal = 0
    for e in ends:
        nodes_x[str(e)] = xVal
        xVal += steps

    # x and y values in list form
    x_values = [nodes_x[n[-1]] for n in node_names]
    y_values = [0.1]*len(x_values)
    
    return x_values, y_values

nodified = nodify(node_names=label)

fig = go.Figure(
    data=[go.Sankey(
    arrangement='snap',
    node = dict(
      pad = 10,
      thickness = 10,
      line = dict(color = "black", width = 0.5),
      label = label,
      hovertemplate='Total of %{value} altcoins %{label}<extra></extra>',
      x=nodified[0],
      y=nodified[1]
    ),
    link = dict(
      arrowlen=20,
      source = sourcedata, # indices correspond to labels, eg A1, A2, A2, B1, ...
      target = targetdata,
      value = valuedata,
      customdata = customdata,
      hovertemplate='Altcoin %{customdata}'+
        '<br />and data %{label}<extra></extra>',
  ))])

fig.update_layout(title_text="Life Cycle of Altcoins", font_size=10,width=1000, height=1000)

fig.show()

In [53]:
ret_df

,year,ticker,benchmark,start_date,start_price,start_price_benchmark,end_date,end_price,end_price_benchmark,factor_period,return_period,factor_period_benchmark,status,over_benchmark,cum_outperformance
1,1,BTC,BTC,2010-07-17,0.05,0.05,2011-07-17,13.16,13.16,265.80,264.80,265.80,Data Available,NaN,NaN
2,2,BTC,BTC,2011-07-17,13.16,13.16,2012-07-17,8.80,8.80,0.67,-0.33,0.67,Data Available,NaN,NaN
3,3,BTC,BTC,2012-07-17,8.80,8.80,2013-07-17,98.50,98.50,11.19,10.19,11.19,Data Available,NaN,NaN
4,4,BTC,BTC,2013-07-17,98.50,98.50,2014-07-17,626.58,626.58,6.36,5.36,6.36,Data Available,NaN,NaN
5,5,BTC,BTC,2014-07-17,626.58,626.58,2015-07-17,279.62,279.62,0.45,-0.55,0.45,Data Available,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,3,ZB,BTC,2020-06-04,0.23,"9,794.55",2021-06-04,0.74,"36,861.50",3.15,2.15,3.76,Data Available,0.84,0.22
4,4,ZB,BTC,2021-06-04,0.74,"36,861.50",2022-06-04,0.13,"29,845.48",0.18,-0.82,0.81,Data Available,0.22,0.05
5,5,ZB,BTC,2022-06-04,0.13,"29,845.48",2023-06-04,0.45,"26,736.21",3.42,2.42,0.90,Data Available,3.82,0.18
6,6,ZB,BTC,2023-06-04,0.45,"26,736.21",2024-06-04,0.45,"26,736.21",1.00,0.00,1.00,Data Available,1.00,0.18


In [64]:
# Assuming your data is stored in a variable called 'data'
# Calculate the average outperformance for each ticker by year
avg_outperformance = ret_df.groupby(['year'])['cum_outperformance'].mean().reset_index()

avg_outperformance
# # Create the line plot with seaborn
# plt.figure(figsize=(20, 10))
# sns.lineplot(x='year', y='over_benchmark', hue='ticker', data=avg_outperformance, ci=None, legend=False)

# # Set plot labels and title
# plt.xlabel('Year', fontsize=16)
# plt.ylabel('Average Outperformance', fontsize=16)
# plt.title('Average Outperformance by Ticker and Year', fontsize=20)

# # Show the plot
# plt.show()

,year,cum_outperformance
0,1,605.14
1,2,684.28
2,3,724.13
3,4,741.81
4,5,741.63
5,6,"850,949.78"
6,7,"850,949.71"
